In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import json
import re
import codecs
import subprocess

from matplotlib import pyplot as plt
import pystruct as pystr


import progressbar


%matplotlib inline

# Load data

In [2]:
with open('slotfilling-data.json', 'r', encoding='UTF8') as file:
    data = json.load(file)

In [3]:
def process_data(data):
    X = [item['chat'] for item in data]
    y = []
    for item in data:
        entities = item['entities']
        y_item = {}
        for entity in entities:
            y_item[entity['title']] = {
                'start_pos': entity['start_pos'],
                'end_pos': entity['end_pos'],
                'text': entity['text']
            }

        y.append(y_item)
    
    return np.array(X), np.array(y)

In [4]:
data, ans = process_data(data)

In [5]:
# Mix the data
perm = np.random.permutation(len(data))
data, ans = data[perm], ans[perm]

In [6]:
possible_slots = set([item for y_item in ans for item in list(y_item.keys())])
possible_slots

{'ВАЛЮТА',
 'ВРЕМЯ_ДАТА_СНЯТИЯ',
 'ЗА_ГРАНИЦЕЙ',
 'МЕСТО_СНЯТИЯ',
 'НАЗВАНИЕ_БАНКА',
 'НОМЕР_ТЕЛЕФОНА',
 'РАЗМЕР_КОМИССИИ',
 'СУММА_СНЯТИЯ',
 'ТАРИФ_КАРТЫ',
 'ТИП_КАРТЫ'}

In [7]:
(data[0], ans[0])

('1: Здравствуйте. У меня на дополнительной карте стоит подтверждение по подписи на чеке, я поменял на пин-код. Сказано, что теперь для подтверждения надо совершить операцию в банкомате. Какую именно?\n2: Можете запросить баланс.\n1: Я сегодня днем запрашивал баланс. До сих пор не подтверждено.\n2: У Вас стоит приоритет ПИН-код.\n1: Имеет значение, в каком банке смотреть? старом/новом\n1: В старом сейчас стоит приоритет подписи.\n2: Вижу, что сейчас приоритет авторизации по Вашей карте – ПИН-код. Чтобы изменения вступили в силу, нужно совершить любую операцию в банкомате (например, снимите наличные или запросите баланс). Учитывайте, что в некоторых точках оплаты у Вас могут запросить подпись. Это зависит от настроек конкретных терминалов.\n1: Может ли зависеть от банкомата? Скажем, если запрошу баланс в другом, то в интернет-банке поменяется отображение приоритета?\n2: Понадобится некоторое время.\n2: Выяснила информацию, Вам нужно попробовать снять наличные в другом банкомате. Чтобы б

# Data preprocessing

In [8]:
#remove special symblos and lower 
def deleteExtraSymbols(line):
    if line:
        return re.sub(' +',' ', re.sub(r'[^А-Яа-я0-9€$ ]', u' ', line).lower().rstrip().strip())
    else:
        return None

In [9]:
clean_data = []
for dialog in data:
    clean_data.append(deleteExtraSymbols(dialog))

In [10]:
# лемматизация
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [11]:
lem_data = []
max_len = 0 # maximum length of the string
for dialog in clean_data:
    lem_dialog = ''
    dialog_words = dialog.split(' ')
    if len(dialog_words) > max_len: max_len = len(dialog_words)
    for word in dialog_words:
        p = morph.parse(word)[0]
        lem_dialog += p.normal_form + ' '
    lem_dialog = lem_dialog[:-1]
    lem_data.append(lem_dialog)

In [12]:
# Приведем ответы к виду ['О', 'О', ..., 'ИМЯ_СЛОТА', ..., 'О', 'О']
counter = 0
labels = []
for i in range(len(clean_data)):
    labels_vec = ['0'] * len(clean_data[i].split(' '))
    for slot in possible_slots:
        try:
            for slot_word in deleteExtraSymbols(ans[i][slot]['text']).split(' '):
                labels_vec[clean_data[i].split(' ').index(slot_word)] = slot
                #print (slot)
        except:
            pass
    labels.append(labels_vec)
    counter += 1

# Vectorize data

In [13]:
#wordList =  [[x.lower() for x in re.findall(r"[\w']+", y)] for y in lem_data]
idx2w = [] 
for dialog in lem_data:
    for word in dialog.split(' '):
        if not word in idx2w:
            idx2w.append(word)

In [14]:
X = []
for dialog in lem_data:
    n_dialog = []
    for word in dialog.split(' '):
        n_dialog.append(idx2w.index(word))
    #for i in range(max_len - len(n_dialog)):
    #    n_dialog.append(10000) # padding
    #lengths.append(len(n_dialog))
    X.append(np.array(n_dialog))
X = np.array(X)

In [52]:
idx2la = []
y_num = []

for line in labels:
    y_line = []
    for slot in line:
        if not slot in idx2la:
            idx2la.append(slot)
        y_line.append(idx2la.index(slot))
    y_num.append(np.array(y_line))
y = np.array(y_num)

In [53]:
#padding
from keras.preprocessing import sequence

X = sequence.pad_sequences(X, maxlen=max_len)
y = sequence.pad_sequences(y, maxlen=max_len)

In [54]:
X_train, y_train, X_test, y_test = X[:6000], y[:6000], X[6000:], y[6000:]

# Solution

In [65]:
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN, GRU, LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers import Convolution1D, MaxPooling1D
from keras.utils import to_categorical

In [67]:
#xin = Input(batch_shape=(batch, timesteps), dtype='int32')
vocab_size = len(idx2w)
n_classes = len(idx2la)
model = Sequential() 
model.add(Embedding(vocab_size, max_len)) # 3dim (batch,time,feat)
model.add(LSTM(100, return_sequences=True))
model.add(TimeDistributed(Dense(n_classes, activation='softmax')))            
model.compile(loss='sparse_categorical_crossentropy',                                   
              optimizer='rmsprop',                                               
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train.reshape(y_train.shape[0], y_train.shape[1], 1), epochs=1, batch_size=50)

Epoch 1/1
3650/6000 [=================>............] - ETA: 510s - loss: 0.0722 - acc: 0.9852

In [53]:
score = model.evaluate(X_test, y_test.reshape(y_test.shape[0], y_test.shape[1], 1), batch_size=50)

500/500 [==============================] - 46s    


In [47]:
# one hot encoding y

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit([[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10]])

y_encoded = []

for line in y:
    y_encoded.append(enc.transform(line.reshape(line.shape[0], 1)).toarray())

y = np.array(y_encoded)